In [129]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Bidirectional, LSTM, Dense, Dropout, Input, 
    Multiply, Activation, Lambda
)
from tensorflow.keras.callbacks import EarlyStopping

In [130]:
class TemporalAttention(tf.keras.layers.Layer):
    def __init__(self, units=50, **kwargs):
        super(TemporalAttention, self).__init__(**kwargs)
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)

    def call(self, inputs):
        hidden_with_time_axis = tf.expand_dims(inputs, axis=1)
        score = self.V(tf.nn.tanh(
            self.W1(inputs) + self.W2(hidden_with_time_axis)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * inputs
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

In [131]:
df_train = pd.read_csv('/home/alvaro/tf_templates/DATA/DailyDelhiClimateTrain.csv', parse_dates=['date'], dayfirst=True).set_index('date')
df_test = pd.read_csv('/home/alvaro/tf_templates/DATA/DailyDelhiClimateTest.csv', parse_dates=['date']).set_index('date')

In [132]:
train = df_train[['meantemp']]
test = df_test[['meantemp']]

In [133]:
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(train)
scaled_test = scaler.transform(test)

In [134]:
length = 90
batch_size = 32
n_features = 1

In [135]:
generator = TimeseriesGenerator(scaled_train, scaled_train, length=length, batch_size=batch_size)
validation_generator = TimeseriesGenerator(scaled_test, scaled_test, length=length, batch_size=batch_size)

In [144]:
inputs = Input(shape=(length, n_features))

In [145]:
x = Bidirectional(LSTM(100, return_sequences=True))(inputs)
x = Dropout(0.3)(x)

In [146]:
x = Bidirectional(LSTM(50, return_sequences=True))(x)
x = Dropout(0.2)(x)

In [147]:
context_vector, attention_weights = TemporalAttention(50)(x)

In [148]:
x = Dense(50, activation='relu')(context_vector)
outputs = Dense(n_features)(x)

In [149]:
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [150]:
early_stop = EarlyStopping(monitor='val_mae', patience=10, restore_best_weights=True)

In [151]:
history = model.fit(generator, epochs=100, validation_data=validation_generator, callbacks=[early_stop])

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - loss: 0.0921 - mae: 0.2420 - val_loss: 0.0596 - val_mae: 0.2132
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0365 - mae: 0.1563 - val_loss: 0.0687 - val_mae: 0.2368
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0366 - mae: 0.1519 - val_loss: 0.0674 - val_mae: 0.2353
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0383 - mae: 0.1576 - val_loss: 0.0287 - val_mae: 0.1392
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0400 - mae: 0.1594 - val_loss: 0.1125 - val_mae: 0.3222
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0349 - mae: 0.1520 - val_loss: 0.1197 - val_mae: 0.3320
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0556 - mae: 0.1960 - val_loss: 0.0851 - val_mae: 0.2760
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0405 - mae: 0.1613 - val_loss: 0.0781 - val_mae: 0.2595
Epoch 9/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - lo

In [152]:
test_predictions = []
first_eval_batch = scaled_train[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(len(scaled_test)):
    current_prediction = model.predict(current_batch, verbose=0)[0]
    test_predictions.append(current_prediction)
    current_batch = np.append(current_batch[:, 1:, :], [[current_prediction]], axis=1)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 4 dimension(s)

In [ ]:
current_batch